In [1]:
from circuit import gen_random_circuits
from circuit.formatter import qiskit_to_my_format_circuit, layered_circuits_to_qiskit, get_layered_instructions
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend, gen_linear_topology, get_linear_neighbor_info, gen_fulllyconnected_topology

from qiskit import QuantumCircuit
import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op_jax import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op_jax import layer_circuit_to_matrix

from scipy.stats import unitary_group

from downstream.synthesis.synthesis_model_pca_unitary_jax import find_parmas, pkl_dump, pkl_load, matrix_distance_squared, SynthesisModel, synthesize
from itertools import combinations
import time
from qiskit import transpile
import random
import cloudpickle as pickle
from circuit.formatter import layered_circuits_to_qiskit, to_unitary
from qiskit.quantum_info import Operator
from jax import vmap
from utils.unitaries import qft_U, grover_U


In [9]:

n_qubits = 3
topology = gen_fulllyconnected_topology(n_qubits)
neigh_info = gen_fulllyconnected_topology(n_qubits)

# topology = gen_linear_topology(n_qubits)
# neigh_info = get_linear_neighbor_info(n_qubits, 1)
synthesis_data_path = f'./temp_data/{n_qubits}_synthesis_data.pkl'
regen = True
if regen:
    backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates=['u'],
                    basis_two_gates=['cz'], divide=False, decoupling=False)


    min_gate, max_gate = 2 * 4**n_qubits - 200, 2 * 4**n_qubits
    dataset = gen_random_circuits(min_gate=100, max_gate=max_gate, gate_num_step=max_gate//50, n_circuits=25,
                                    two_qubit_gate_probs=[2, 5], backend=backend, reverse=False, optimize=True, multi_process=True)


    upstream_model = RandomwalkModel(1, 20, backend)
    upstream_model.train(dataset, multi_process=True, remove_redundancy=False)
    synthesis_model = SynthesisModel(upstream_model, f'synthesis_{n_qubits}')
    data = synthesis_model.construct_data(dataset, multi_process = True)
    # synthesis_model.construct_model(data)
    synthesis_model.save()
    
    with open(synthesis_data_path, 'wb') as f:
        pickle.dump(data, f)
        
    Us, Vs = data
    
    print(np.allclose(Us[0] @ Us[0].T.conj(), np.eye(2**n_qubits)))
    print(matrix_distance_squared(Us[0] @ Us[0].T.conj(), np.eye(2**n_qubits)))
    print(matrix_distance_squared(Us[0], Us[0]))
else:
    synthesis_model: SynthesisModel = SynthesisModel.load(f'synthesis_{n_qubits}')
    backend: Backend = synthesis_model.backend

    with open(synthesis_data_path, 'rb') as f:
        Us, Vs = pickle.load(f)

(_gen_random_circuits_remote pid=22161) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: divide by zero encountered in det
(_gen_random_circuits_remote pid=22161)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=22161) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: invalid value encountered in det
(_gen_random_circuits_remote pid=22161)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=22162) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: divide by zero encountered in det
(_gen_random_circuits_remote pid=22162)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=22162) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: invalid value encountered in det
(_gen_random_circu

(remote_train pid=22164) train:0/105, 315th offest
(remote_train pid=22162) train:0/105, 105th offest
(remote_train pid=22161) train:0/105, 210th offest
(remote_train pid=22157) train:0/105, 630th offest
(remote_train pid=22156) train:0/105, 525th offest
(remote_train pid=22160) train:0/105, 420th offest
(remote_train pid=22163) train:0/105, 0th offest
(remote_train pid=22159) train:0/105, 945th offest
(remote_train pid=22158) train:0/105, 735th offest
(remote_train pid=22165) train:0/105, 840th offest
(remote_train pid=22164) train:100/105, 315th offest
(remote_train pid=22162) train:100/105, 105th offest
(remote_train pid=22161) train:100/105, 210th offest
(remote_train pid=22163) train:100/105, 0th offest
(remote_train pid=22165) train:100/105, 840th offest
(remote_train pid=22158) train:100/105, 735th offest
(remote_train pid=22156) train:100/105, 525th offest
(remote_train pid=22160) train:100/105, 420th offest
(remote_train pid=22157) train:100/105, 630th offest
count path
random

100%|██████████| 1050/1050 [01:15<00:00, 13.87it/s]


len(Us) =  227390 len(gate_vecs) =  227390
True
0.0
0.0


In [3]:
U = unitary_group.rvs(2**n_qubits)
dists = vmap(matrix_distance_squared, in_axes = (None, 0))(U, Us)
np.argmin(dists), np.min(dists)


(Array(8586, dtype=int64), Array(0.63913796, dtype=float64))

In [4]:
matrix_distance_squared(unitary_group.rvs(2**n_qubits), unitary_group.rvs(2**n_qubits))

Array(0.96265402, dtype=float64)

In [5]:
dists.size

9450

In [6]:
_n_qubits = 6
_Us = np.array([unitary_group.rvs(2**_n_qubits) for i in range(100000)])

In [7]:
U = unitary_group.rvs(2**_n_qubits)
dists = vmap(matrix_distance_squared, in_axes = (None, 0))(U, _Us)
np.argmin(dists), np.min(dists)

(Array(81145, dtype=int64), Array(0.94706898, dtype=float64))

In [8]:
U.shape, _Us.shape

((64, 64), (100000, 64, 64))